ALL IMPORT 

In [1]:
import time
import pandas as pd
import csv
import os
import string as st
import numpy as np
import itertools    
from stemming.porter2 import stem
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import multiprocessing
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.dummy import DummyClassifier
from joblib import dump

stopWords = stopwords.words("english")
cores = multiprocessing.cpu_count()

In [ ]:
SETTING ALL DATA

In [ ]:
PATH_TRAINING = "blog/train/"
PATH_TEST = "blog/test/test1/"
training_file = os.listdir(PATH_TRAINING)
test_file = os.listdir(PATH_TEST)

In [ ]:
def saving_by_author(directory,PATH_SOURCE,destinationfile):
    data_frame=[]
    count=0
    for file in directory:
        all_text= None
        count=count+1
        print(count)
        path = PATH_SOURCE + file
        df = pd.read_csv(path, header=0, sep=',', quotechar='"', names=['code', 'gender', 'age', 'sign', 'text'])
        if df.empty:
            print('here')
            ff =open(path)
            reader = csv.reader(ff)
            for row in reader:
                df=df.append(pd.DataFrame({ 'code': row[0], 'gender': row[1],'age':row[2],'sign': row[3],'text': row[4]},index=[0]))
        all_text = df.groupby('code')['text'].apply(' '.join).reset_index()
        text = list(all_text['text'])[0]
        df_to_append=pd.DataFrame({ 'code': df['code'][0], 'gender': df['gender'][0],'age':df['age'][0],'sign': df['sign'][0],'text': text},index=[0])
        data_frame.append(df_to_append)
    new_set = pd.concat(data_frame,ignore_index=True)
    new_set.to_csv(destinationfile,index=False)
    print(len(new_set))
    return new_set



In [ ]:
train_set = pd.read_pickle("clean_trainset.pkl")
test_set = pd.read_pickle("clean_testset.pkl")

In [ ]:
train_set.info()

In [ ]:
test_set.info()

In [24]:
quiz_set=pd.read_pickle("blind_test.pkl")

In [25]:
quiz_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   code                   2000 non-null   object
 1   text                   2000 non-null   object
 2   tokenized_text         2000 non-null   object
 3   tokenized_text_string  2000 non-null   object
dtypes: object(4)
memory usage: 62.6+ KB


In [2]:
tokens_train = pd.Series(train_set['tokenized_text']).values
tokens_test = pd.Series(test_set['tokenized_text']).values

tokens_train_gender = pd.Series(train_set['gender']).values
tokens_test_gender = pd.Series(test_set['gender']).values

tokens_string_train = pd.Series(train_set['tokenized_text_string']).values
tokens_string_test = pd.Series(test_set['tokenized_text_string']).values

all_female=pd.Series(train_set[train_set['gender']==1]['tokenized_text']).values
all_male=pd.Series(train_set[train_set['gender']==0]['tokenized_text']).values

all_result={}

GETTING OUR BENCHMARK

In [5]:
clf= DummyClassifier(strategy='most_frequent')

In [6]:
clf.fit(train_set['tokenized_text_string'],train_set['gender'])

DummyClassifier(strategy='most_frequent')

In [8]:
pre = clf.predict(test_set['tokenized_text_string'])

In [10]:
print(accuracy_score(test_set['gender'],pre))

0.5060189428658722


WORD2VEC

In [ ]:
def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return np.zeros((model.vector_size,))

def mean_vectors(phrase, model):
    a=[get_vect(w, model) for w in phrase]
    return sum(a)/len(a)

def word2vec_features(X, model):
    feats = np.vstack([mean_vectors(p, model) for p in X])
    return feats

In [ ]:
w2v_model = Word2Vec(sg=0,hs=1,negative=0,sorted_vocab=1,workers=cores-1)
w2v_model.build_vocab(train_set['tokenized_text'].tolist(), progress_per=10000)
w2v_model.train(train_set['tokenized_text'].tolist(), total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
w2v_model.init_sims(replace=True)
w2v_model.save('w2vmodel.model')

In [ ]:
model = Word2Vec.load("w2vmodel.model")

In [ ]:
wv_train = word2vec_features(tokens_train, model)
wv_test = word2vec_features(tokens_test, model)

TRAINING word2Vec

In [ ]:
svm_word2vec = SVC()

start_time_train_svm_word2vec= time.time()
svm_word2vec.fit(wv_train, tokens_train_gender)
end_time_train_svm_word2vec= time.time()

In [ ]:
start_time_test_svm_word2vec= time.time()
predict_svm_word2vec=svm_word2vec.predict(wv_test)
end_time_test_svm_word2vec= time.time()

predict_score_svm_word2vec=our_score(predict_svm_word2vec,tokens_test_gender)
all_result['svm_word2vec']=[predict_score_svm_word2vec,(end_time_train_svm_word2vec-start_time_train_svm_word2vec),(end_time_test_svm_word2vec-start_time_test_svm_word2vec)]

In [ ]:
all_result

In [ ]:
lr_word2vec = LogisticRegression()

start_time_train_lr_word2vec= time.time()
lr_word2vec.fit(wv_train, tokens_train_gender)
end_time_train_lr_word2vec= time.time()

In [ ]:
start_time_test_lr_word2vec= time.time()
predict_lr_word2vec=lr_word2vec.predict(wv_test)
end_time_test_lr_word2vec= time.time()

predict_score_lr_word2vec=our_score(predict_lr_word2vec,tokens_test_gender)
all_result['lr_word2vec']=[predict_score_lr_word2vec,(end_time_train_lr_word2vec-start_time_train_lr_word2vec),(end_time_test_lr_word2vec-start_time_test_lr_word2vec)]


In [ ]:
all_result

In [ ]:
km_word2vec = KMeans(n_clusters=2)
start_time_train_svm_word2vec= time.time()
km_word2vec.fit(wv_train)
end_time_train_svm_word2vec= time.time()

In [ ]:
start_time_test_km_word2vec= time.time()
predict_km_word2vec=km_word2vec.predict(wv_test)
end_time_test_km_word2vec= time.time()

predict_score_km_word2vec =our_score(predict_km_word2vec,tokens_test_gender)
all_result['km_word']=[predict_score_km_word2vec,(end_time_train_svm_word2vec-start_time_train_svm_word2vec),(end_time_test_km_word2vec-start_time_test_km_word2vec)]


In [ ]:
all_result

In [ ]:
mlp_word2vec = MLPClassifier()

start_time_train_mlp_word2vec= time.time()
mlp_word2vec.fit(wv_train, tokens_train_gender)
end_time_train_mlp_word2vec= time.time()


In [ ]:
start_time_test_mlp_word2vec= time.time()
predict_mlp_word2vec=mlp_word2vec.predict(wv_test)
end_time_test_mlp_word2vec= time.time()

predict_score_mlp_word2vec=accuracy_score(tokens_test_gender,predict_mlp_word2vec)
all_result['mlp_word2vec']=[predict_score_mlp_word2vec,(end_time_train_mlp_word2vec-start_time_train_mlp_word2vec),(end_time_test_mlp_word2vec-start_time_test_mlp_word2vec)]


In [ ]:
all_result

TF-IDF

In [14]:
vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(tokens_string_train)

In [15]:
tfidf_test = vectorizer.transform(tokens_string_test)

TRAINING tf-idf

In [ ]:
svm_tfidf = SVC()

start_time_train_svm_tfidf= time.time()
svm_tfidf.fit(tfidf_train, tokens_train_gender)
end_time_train_svm_tfidf= time.time()

In [ ]:
start_time_test_svm_tfidf= time.time()
predict_svm_tfidf=svm_tfidf.predict(tfidf_test)
end_time_test_svm_tfidf= time.time()

predict_score_svm_tfidf=accuracy_score(tokens_test_gender,predict_svm_tfidf)
all_result['svm_tfidf']=[predict_score_svm_tfidf,(end_time_train_svm_tfidf-start_time_train_svm_tfidf),(end_time_test_svm_tfidf-start_time_test_svm_tfidf)]


In [ ]:
all_result

In [ ]:
lr_tfidf = LogisticRegression()

start_time_train_lr_tfidf= time.time()
lr_tfidf.fit(tfidf_train, tokens_train_gender)
end_time_train_lr_tfidf= time.time()

In [ ]:
start_time_test_lr_tfidf= time.time()
predict_lr_tfidf=lr_tfidf.predict(tfidf_test)
end_time_test_lr_tfidf= time.time()

predict_score_lr_tfidf=accuracy_score(tokens_test_gender,predict_lr_tfidf)
all_result['lr_tfidf']=[predict_score_lr_tfidf,(end_time_train_lr_tfidf-start_time_train_lr_tfidf),(end_time_test_lr_tfidf-start_time_test_lr_tfidf)]


In [ ]:
all_result

In [ ]:
km_tfidf = KMeans(n_clusters=2)
start_time_train_km_tfidf= time.time()
km_tfidf.fit(tfidf_train)
end_time_train_km_tfidf= time.time()

In [ ]:
start_time_test_km_tfidf= time.time()
predict_km_tfidf=km_tfidf.predict(tfidf_test)
end_time_test_km_tfidf= time.time()

predict_score_km_tfidf =accuracy_score(tokens_test_gender,predict_km_tfidf)
all_result['km_tfidf']=[predict_score_km_tfidf,(end_time_train_km_tfidf-start_time_train_km_tfidf),(end_time_test_km_tfidf-start_time_test_km_tfidf)]


In [ ]:
all_result

In [ ]:
mlp_tfidf = MLPClassifier()

start_time_train_mlp_tfidf= time.time()
mlp_tfidf.fit(tfidf_train, tokens_train_gender)
end_time_train_mlp_tfidf= time.time()


In [ ]:
start_time_test_mlp_tfidf= time.time()
predict_mlp_tfidf=mlp_tfidf.predict(tfidf_test)
end_time_test_mlp_tfidf= time.time()

predict_score_mlp_tfidf=accuracy_score(tokens_test_gender,predict_mlp_tfidf)
all_result['mlp_tfidf']=[predict_score_mlp_tfidf,(end_time_train_mlp_tfidf-start_time_train_mlp_tfidf),(end_time_test_mlp_tfidf-start_time_test_mlp_tfidf)]


In [ ]:
all_result

MALE vs FEMALE

In [ ]:
def print_n_first(n,d):
    x = itertools.islice(d.items(), 0, n)
    for key, value in x:
        print(key)
        print(value)
        print('-----')

In [ ]:
def merge_list(list_to_merge):
    list_to_return=[]
    for sublist in list_to_merge:
        list_to_return=list_to_return+eval(sublist)
    return list_to_return

In [ ]:
male=merge_list(all_male)
mal=Counter(male)

In [ ]:
female =merge_list(all_female)
fem = Counter(female)

In [ ]:
sorted_dict_fem={k: v for k, v in sorted(dict(fem).items(), key=lambda item: item[1],reverse=True)}
sorted_dict_mal={k: v for k, v in sorted(dict(mal).items(), key=lambda item: item[1],reverse=True)}

In [ ]:
print_n_first(50,sorted_dict_fem)
print_n_first(50,sorted_dict_mal)

Hyper-paramètre & Random search

In [ ]:
tuned_parameters= [
  {'kernel': ['rbf'],
  'gamma': [1,0.1,0.01,0.001,1000,1/len(tokens_string_train)],
  'C': [0.1,1,10,100,1000],
  'max_iter' : [-1,1000,500,200]},
  
  {'kernel': ['linear'],
   'C': [0.1,1,10,100,1000],
  'max_iter' : [-1,1000,500,200]},
]

metric = 'f1_macro'

In [ ]:
grid_search_svm = RandomizedSearchCV(
    SVC(), tuned_parameters, n_iter=12, scoring=metric)
grid_search_svm.fit(tfidf_train, tokens_train_gender);
print('Finished!')

In [ ]:
print(grid_search_svm.best_params_)

In [17]:
svm_tfidf_best = SVC(max_iter= -1, kernel= 'rbf', gamma= 'scale', C= 100)

start_time_train_svm_tfidf_best= time.time()
svm_tfidf_best.fit(tfidf_train, tokens_train_gender)
end_time_train_svm_tfidf_best= time.time()

In [18]:
start_time_test_svm_tfidf_best= time.time()
predict_svm_tfidf_best=svm_tfidf_best.predict(tfidf_test)
end_time_test_svm_tfidf_best= time.time()

predict_score_svm_tfidf_best=accuracy_score(tokens_test_gender,predict_svm_tfidf_best)

In [21]:
all_result['svm_tfidf_best']=[predict_score_svm_tfidf_best,(end_time_train_svm_tfidf_best-start_time_train_svm_tfidf_best),(end_time_test_svm_tfidf_best-start_time_test_svm_tfidf_best)]


In [ ]:
all_result

Quiz

In [26]:
tokens_quiz = pd.Series(quiz_set['tokenized_text']).values
tokens_string_quiz = pd.Series(quiz_set['tokenized_text_string']).values

In [27]:
tfidf_quiz = vectorizer.transform(tokens_string_quiz)

In [28]:
predict_svm_tfidf_best=svm_tfidf_best.predict(tfidf_quiz)

In [31]:
append_quiz_set=pd.read_pickle("blindTest_age.pkl")

In [33]:
append_quiz_set['gender']=predict_svm_tfidf_best

In [36]:
append_quiz_set['gender'] = append_quiz_set['gender'].map({1:'female', 0:'male'})


In [40]:
append_quiz_set=append_quiz_set[['code','gender','age','tokenized_text','tokenized_text_string']]

In [43]:
append_quiz_set.to_pickle("blindTest_age_gender.pkl")

In [49]:
append_quiz_set.to_csv("blindTest_age_gender.csv")